In [150]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
sns.set()
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [152]:
KAGGLE_DIR = 'data/'
#KAGGLE_DIR = '../input/'

import os
print(os.listdir(KAGGLE_DIR))

['test_images', 'breed_labels.csv', '.DS_Store', 'test', 'train_metadata', 'color_labels.csv', 'test_sentiment', 'test_metadata', 'train_sentiment', 'train', 'train_images', 'state_labels.csv']


In [153]:

train=pd.read_csv(KAGGLE_DIR+'train/train.csv')
test=pd.read_csv(KAGGLE_DIR+'test/test.csv')
breed_labels=pd.read_csv(KAGGLE_DIR+'breed_labels.csv')
breed_labels['tipo_animal']= np.where(breed_labels['Type']==1,'Dog','Cat')  #Difference between animals
color_labels=pd.read_csv(KAGGLE_DIR+'color_labels.csv')
color_labels.set_index('ColorID',inplace=True)
state_labels=pd.read_csv(KAGGLE_DIR+'state_labels.csv')
full=train.append(test)

In [154]:
print(train.shape)
print(test.shape)
print(full.shape)

(14993, 24)
(3948, 23)
(18941, 24)


### Color

In [155]:
full['SecondaryColors']=full['Color2']+full['Color3']
full['MonoColor']=np.where(full['SecondaryColors'],1,0)

### State

In [156]:
full['State_3']=full.State.map(lambda x: "{:05}".format(x)[:3])
full['State_4']=full.State.map(lambda x: "{:05}".format(x)[:4])

### Age

In [157]:
full['LessThanAYear']=np.where(full['Age']<12,1,0)

### Comentario

In [158]:
full['Description_Count'] = full['Description'].str.len()
full['hasName'] = (full.Name.isna()).astype(int)


full['Name_Len']=full['Name'].str.len()

### Pet size

In [159]:
full['BigSizePet']= np.where(full['MaturitySize']>2,1,0)

### Health

In [160]:
full['isHealthy']= np.where(full['Health']>1,1,0)

### Dewormed & Vaccinated

In [161]:
full['isDewormed']= np.where(full['Dewormed']>1,1,0)
full['isVaccinated']= np.where(full['Vaccinated']>1,1,0)

## Unico o camada

In [162]:
full['isLonely']=np.where(full['Quantity']>1,1,0)

### Free

In [163]:
full['isFree']=np.where(full['Fee']>0,1,0)

# Mean Encodings

In [164]:
#Mean Encoding Function
def Target_encoding(df,element,target):
    #create aux    
    aux_df = pd.DataFrame(df.groupby(element)[target].mean())

    #change name
    aux_df[element+'_ENCODED'] = aux_df[target]
    aux_df= aux_df.drop(target, axis=1)

    #join
    df = df.join(aux_df, how = 'left', on = element) 
    return df


In [165]:
def splited_target_encoding(df,query_train,query_test,element,target):
    entrenamiento=df.query(query_train)
    testeo=df.query(query_test)
    
    entrenamiento=Target_encoding(entrenamiento,element,target)
    
    inputZIP1=element
    inputZIP2=element+'_ENCODED'
    
    mydict=dict(zip(entrenamiento[inputZIP1],entrenamiento[inputZIP2]))
    testeo[element+'_ENCODED']=testeo[element].map(mydict)
    return entrenamiento.append(testeo)


In [166]:
#Zip Encodings
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','State_4','AdoptionSpeed')
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','State_3','AdoptionSpeed')

In [167]:
labels = [1,2,3,4,5]
full['AgeBins'] = pd.cut(full['Age'],5,labels=labels)
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','AgeBins','AdoptionSpeed')

In [168]:
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','Breed1','AdoptionSpeed')

In [169]:
full['FullBlack']=np.where((full['Color1']==1) & (full['MonoColor']==0),1,0)
full['FullWhite']=np.where((full['Color1']==7) & (full['MonoColor']==0),1,0)

In [170]:
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','RescuerID','AdoptionSpeed')

In [171]:
labels = [1,2,3,4,5]
full['FeeBins'] = pd.cut(full['Fee'],5,labels=labels)
full=splited_target_encoding(full,'AdoptionSpeed!="NaN"','AdoptionSpeed=="NaN"','FeeBins','AdoptionSpeed')

In [172]:
full.shape

(18941, 48)

In [173]:
full.RescuerID_ENCODED

0       2.000000
1       0.000000
2       2.368192
3       2.640000
4       1.970149
5       2.000000
6       1.800000
7       2.333333
8       2.714286
9       4.000000
10      2.640000
11      2.333333
12      2.019231
13      1.000000
14      2.000000
15      4.000000
16      2.000000
17      4.000000
18      2.640000
19      3.000000
20      2.800000
21      4.000000
22      4.000000
23      2.603175
24      1.363636
25      2.639344
26      1.000000
27      2.333333
28      3.000000
29      2.181818
          ...   
3918         NaN
3919         NaN
3920         NaN
3921         NaN
3922         NaN
3923         NaN
3924         NaN
3925         NaN
3926         NaN
3927         NaN
3928         NaN
3929         NaN
3930         NaN
3931         NaN
3932         NaN
3933         NaN
3934         NaN
3935         NaN
3936         NaN
3937         NaN
3938         NaN
3939         NaN
3940         NaN
3941         NaN
3942         NaN
3943         NaN
3944         NaN
3945         N

# Machine Learning

In [174]:
#Lets fill de na
full.Description_Count.fillna(0,inplace=True)
full.Name_Len.fillna(0,inplace=True)
# Spliting the data
test=full.query('AdoptionSpeed =="NaN"')
train=full.query('AdoptionSpeed !="NaN"')
target=train['AdoptionSpeed']
train.drop(columns=['AdoptionSpeed','Description','Name','PetID','RescuerID','RescuerID_ENCODED'],inplace=True)
test.drop(columns=['AdoptionSpeed','Name', 'RescuerID', 'Description', 'PetID','RescuerID_ENCODED'],inplace=True)

In [175]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# Seed for reproducability
seed = 12345
np.random.seed(seed)


In [176]:
# Metric used for this competition (Quadratic Weigthed Kappa aka Cohen Kappa Score)
def metric(y1,y2):
    return cohen_kappa_score(y1,y2, weights='quadratic')

### Random Forest

In [177]:
tic=time.time()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
toc=time.time()
print('The time elapsed is {}s'.format(np.round(np.abs(tic-toc))))

{'n_estimators': [200, 650, 1100, 1550, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 35, 60, 85, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True]}
The time elapsed is 0.0s


In [178]:
#Has nans?
train.isna().sum().sum()

0

In [179]:
rf=RandomForestClassifier(n_estimators=1100,min_samples_split=2,min_samples_leaf=1,max_features='auto',max_depth=60,bootstrap=True)
rf.fit(train,target)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [180]:
# Measure of performance 
# Useful for checking overfitting, performance, etc.
metric(rf.predict(train), target)

0.9989653186984565

### Extratrees

In [181]:
extra_trees = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=83, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=225, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)



In [182]:
extra_trees.fit(train,target)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=83, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=225, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [183]:
metric(extra_trees.predict(train), target)

0.6411256394135593

### CatBoost

In [184]:
from catboost import CatBoostClassifier

In [185]:
cb = CatBoostClassifier(iterations=2, learning_rate=1, depth=8, loss_function='MultiClass')
cb.fit(train,target);

0:	learn: -1.4193188	total: 235ms	remaining: 235ms
1:	learn: -1.3749724	total: 405ms	remaining: 0us


In [186]:
metric(cb.predict(train), target)

0.2939917054500589

### Adaboost

In [187]:
adab=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.3, n_estimators=200, random_state=None)

In [188]:
adab.fit(train,target)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.3, n_estimators=200, random_state=None)

In [189]:
# Measure of performance 
# Useful for checking overfitting, performance, etc.
metric(adab.predict(train), target)

0.36027630702137936

### GaussianNB

In [190]:
from sklearn.naive_bayes import GaussianNB
gauss=GaussianNB()
gauss.fit(train,target)
metric(gauss.predict(train), target)

0.2414062624902521

### Performance of the models

In [191]:
print('Random Forest score: ', metric(rf.predict(train), 
                                target))
print('Extra Trees score:   ', metric(extra_trees.predict(train), 
                                target))
print('Adaboost score:      ', metric(adab.predict(train), 
                                target))
print('Catboos score:       ', metric(cb.predict(train), 
                                target))
print('GaussianNB score:    ', metric(gauss.predict(train), 
                                target))

Random Forest score:  0.9989653186984565
Extra Trees score:    0.6411256394135593
Adaboost score:       0.36027630702137936
Catboos score:        0.2939917054500589
GaussianNB score:     0.2414062624902521


# Ensambling

In [193]:
test.Breed1_ENCODED.fillna(0,inplace=True)
test.isna().sum().sum()


0

In [194]:
# Get predictions
predictions1 = rf.predict(test)
predictions2 = extra_trees.predict(test)
predictions3 = adab.predict(test)
predictions4 = gauss.predict(test)
predictions5 = cb.predict_proba(test)[:, 1]

# Combine predictions
final_predictions = []
# Get average of predictions
for pred in zip(predictions1, predictions2, predictions3, predictions4,predictions5):
    final_predictions.append(int(round((sum(pred)) / 5, 0)))

In [195]:
# Compare predictions
prediction_df = pd.DataFrame({'Random Forest' : predictions1,
                              'Extra Trees'   : predictions2,
                              'Adaboost'      : predictions3,
                              'GaussianNB'    : predictions4,
                              'Catboost'      : predictions5,
})

prediction_df.head()

,Random Forest,Extra Trees,Adaboost,GaussianNB,Catboost
0,2.0,2.0,2.0,2.0,0.211127
1,4.0,4.0,4.0,4.0,0.093984
2,4.0,4.0,4.0,1.0,0.045199
3,4.0,4.0,4.0,2.0,0.126196
4,4.0,4.0,4.0,2.0,0.054236


### Submission

In [196]:
# Get and store predictions

test=pd.read_csv(KAGGLE_DIR+'test/test.csv')
submission_df = pd.DataFrame(data={"PetID" : test["PetID"], 
                                   "AdoptionSpeed" : final_predictions})


In [197]:
submission_df['AdoptionSpeed']=submission_df['AdoptionSpeed'].astype('int32');
submission_df.to_csv("sample_submission.csv", index=False)

In [198]:
submission_df.head()

,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,3
2,72000c4c5,3
3,e147a4b9f,3
4,43fbba852,3
